In [1]:
import json
import os
import glob
import shutil

def convert_json_to_txt_and_copy_images(input_folder, output_folder_txt, output_folder_images):
    # Ensure output folders exist
    os.makedirs(output_folder_txt, exist_ok=True)
    os.makedirs(output_folder_images, exist_ok=True)

    # Iterate over all JSON files in the input folder
    for json_file in glob.glob(os.path.join(input_folder, '*.json')):
        with open(json_file) as f:
            data = json.load(f)

        # Prepare to collect txt lines
        txt_lines = []
        image_width = data['imageWidth']
        image_height = data['imageHeight']

        for shape in data['shapes']:
            label = shape['label']
            # Calculate coordinates and dimensions
            x_min, y_min = shape['points'][0]
            x_max, y_max = shape['points'][1]
            x_center = ((x_min + x_max) / 2) / image_width
            y_center = ((y_min + y_max) / 2) / image_height
            width = (x_max - x_min) / image_width
            height = (y_max - y_min) / image_height

            # Format line and add to list
            line = f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            txt_lines.append(line)

        # Construct the output txt filename based on the JSON filename
        txt_filename = os.path.splitext(os.path.basename(json_file))[0] + '.txt'
        txt_filepath = os.path.join(output_folder_txt, txt_filename)

        # Write lines to txt file
        with open(txt_filepath, 'w') as txt_file:
            for line in txt_lines:
                txt_file.write(line + '\n')

        # Copy the corresponding image file
        image_filename = os.path.splitext(os.path.basename(json_file))[0] + '.jpg'
        image_filepath = os.path.join(input_folder, image_filename)
        if os.path.exists(image_filepath):
            shutil.copy2(image_filepath, os.path.join(output_folder_images, image_filename))


input_folder = './data/parking_lots_labeled'  
output_folder_labels = './datasets/ds8/train/labels' 
output_folder_images = './datasets/ds8/train/images' 
convert_json_to_txt_and_copy_images(input_folder, output_folder_labels, output_folder_images)
